In [1]:
import pandas as pd
import string
import re
import datetime
import collections
import math
import heapq
import numpy as np
from tqdm import tqdm
import pickle
import operator

## marks lists


In [2]:
punctuation_marks = [')','(','>','<',"؛","،",'{','}',"؟",':',"-", '»', '"', '«', '[', ']','"','+','=','?']
marks = ['/','//', '\\','|','!', '%', '&','*','$', '#','؟', '*','.','_' ]
alphabet_string_lower = string.ascii_lowercase
alphabet_string_upper = string.ascii_uppercase
english_char =  list(alphabet_string_lower) + list(alphabet_string_upper)


sep_list = [" ", '\xad', '\u200e','\u200f', '\u200d', '\u200d', '\u200d'] + marks

# stop_words1 =['به', 'و', 'در', 'با', 'این', 'شد', 'را', 'که', 'از', 'که', 'این', 'با', 'است', 'برای', 'آن', 'یک', 'خود', 'تا', 'کرد', 'بر', 'هم', 'نیز', 'هزار', 'ریال', 'بود']
stop_words = ["برای", "پس", "سپس", "تا", "از", "که", "و", "به", "را", "با", "بر", "در", "اینکه", "این", "ان", "چرا",
              "شاید", "انها", "چون", "انطور", "اینطور", "انچه", "آخ", "آخر", "آخرها", "اخه", "آره", "آری", "انان",
              "اگر", "درباره", "خیلی", "توی", "بلکه", "بعضی", "بعدا", "باید", "البته", "الان", "اصلا", "اساسا",
              "احتمالا", "اخیرا", "ایا", "اهان", "امرانه", "ان گاه", "انرا", "انقدر", "اساسا", "بنابراین", "هرحال",
              "بی‌انکه", "به‌قدری", "بیشتر", "کمتر", "تر", "ترین", "تو", "چند", "چندین", "چه", "خیلی", "دیگر", "دیگه",
              "طبیعتا", "عمدا", "گاهی", "نیز", "او", "تو", "من", "ما", "طور", "دو", "هر", "همه", "ایم", "اند", "اید",
              "است", "هست", "زدگی", "خصوصا", "ن", "ب", "کسی", "چیزی", "بالاخره", "وقتی", "زمانی", "می"]

## stop_words

In [3]:
# stop_word_data_20k = ['', 'پیام', 'خواه','دار', 'شده', '۵', 'گزار', 'ها', 'گیر', 'قرار', 'انت', '۴', 'حضور', 'وی', 'می',  'ان', 'بین', 'سال', '۷', 'گفت', 'کار', 'ی', 'کرده', 'اما', 'ادامه', 'شو', 'روز', 'کن', 'گذشته', 'داشت', 'های', 'باز', 'انجام', 'داد', '۳', 'باید', 'اینکه', 'پیش', 'بیش', 'دو', 'باش', 'توان', 'دیگر', '۲', 'عنو',  'امروز', '۹', '۸', 'پای', 'مورد', 'صورت', 'مردم', 'ب', 'هر', 'ما', 'ایر', 'رو', 'اظهار', '۱', 'افزود', '–', '۶', 'ا', 'ر', 'ث',  '۰', 'ش', 'د', 'ژ', 'ک', 'ع', 'ه', 'ج', 'ز', 'م', 'ت', '٢', '٣', '٧', 'ق', 'ل', 'ن', '@', ',', '\u2066', '٩', '٤', '٥', '٨', '١', '٦', 'س', 'ص', 'ء', 'ف', "'", 'ö', 'گ', '》', 'ط', 'پ', '…', 'خ', 'ê', 'ح', '\u2067', '\u2069', 'ۗ', '•', '٪', '\u200b', '×', 'ۚ', 'ۖ', 'چ', '"', '\u2063', 'ذ', '\uf0d8', '۔', 'ü', 'ﻭ', 'غ', '️', 'ﷲ', '\u202c', '\u202a', 'ﻫ', 'ۙ', 'ض', '·', '¬', ';', 'خبر گزاری','گزازش','خبرنگار',]
# # stop_word_data_17k = ['', 'درصد', 'وی', 'کن', 'توان', 'پیام', 'ما', 'گفت', 'دولت', 'خواه', 'باید', 'اینکه','ان', 'تولید', 'دار', 'اگر', 'هستند', , 'شو', 'پیش', 'وجود','ایر', 'شده', 'کار', 'برخی', 'رو', 'بین', 'توسط', 'گیر', 'نظر', 'باش', 'همچنین', 'سال', 'دو', 'داشت', 'انجام', 'افزای', 'اما', 'هر', 'حال', 'قرار', 'مورد', 'افزود', 'گرفته',  'انها', 'انت', 'دلیل', 'ب', 'ادامه', 'می', 'شرکت', 'اساس', 'پای', 'گزار', 'کرده', 'ساز', 'بخش', 'بیش', 'داد', 'موضوع', 'روز', 'یا', 'شرایط', 'بوده',  'همین', 'کاهش', 'گذشته', 'توجه', 'یکی', 'امروز', 'ماه', 'زار', 'اقدام', '۵', 'داشته', '۲', 'عنو', 'داده', 'های', 'دیگر', 'نسبت', 'بیان', 'همه', '۸', 'اعلام',  'نقل', 'پس', '۷', 'میلیون', '۹', 'ر', '۶', '۴', '۳', 'ه', '۰', 'گروه', 'اشاره', '۱۳۹۹', '۱', 'ج', 'مردم', 'استفاده', 'ی', 'ت', '٥', '٢', '١', '٣', '٦', 'د', '–', 'ژ', 'ص', 'ع', 'م', 'پ', 'ا', '٤', 'ش', '٧', '٩', 'ک', 'ز', 'ف', 'ح', '’', 'ن', ',', '٨', "'", 'چ', '٠', '•', '·', '\u200b', 'غ', '…', 'ل', 'خ', 'ذ', 'ث', '\uf0d8', 'س', '@', 'ق', 'ء', 'ط', 'گ', ';', '٬', 'ۗ', '\uf0fc', '—', 'ñ', 'á', 'ﻫ', '×', '−', '±', '"', '﴾', '°', 'ﻭ',  '🔹', 'ۚ', 'ۖ', '\u202b', 'ş', 'é', 'ة', 'ۙ']
# stop_word_data_11k = ['', 'پیام', 'اینکه', 'ایر', 'شده', 'کن', 'باش', 'کشور', 'های', 'کار', 'روز', 'اعلام', 'گزار', 'همچنین', 'سال', 'کرده', 'داد', 'انت', 'عنو', '۳', 'حضور', 'خواه', 'وی', 'افزود', 'گفت', 'بیان', 'توان', 'شو', 'بخش', 'باید', 'دار', 'داشت', 'ان', 'توجه', 'مورد', 'دو', 'انجام', '۲', 'رو', 'هر', 'یا', 'صورت', 'هستند', 'اما', 'ی', '٢', 'گذشته', '۴', 'قرار', 'بیش', 'دیگر', 'ادامه', 'امروز', '۰', '۶', '۵', '۹', '۷', '۸', '۱', 'ا', '٣', 'ب', 'س', '–', '٨', 'ج', '٧', 'پ', "'", 'ز', 'ر', 'ه', 'ع', 'ء', 'م', '\u202c', 'ش', '٦', 'ن', '٩', '@', 'ۚ', '٤', 'د', 'ص', 'ک', '\u202b', 'ث', 'چ', 'ل', 'ط', 'ح', 'ف', 'ت', '…', '٥', '\u2067', '١', 'ذ', 'ق', '—', '٠', ',', 'غ', '\ufeff', '٪', '"', '\u2069', '•', 'ۗ', '’', 'ژ', 'خ', 'и', 'О', 'с', 'б', 'Я', 'С', 'в', '۔', '·','بر','تا','در','به','از', 'گ', 'Ö', 'é', ';', '\u206b', '》', '⠀', '\u202a', 'ۖ', '❇', 'ﻫ'] + stop_words1
# stop_words = set(stop_word_data_20k + stop_word_data_11k)

## regex  patterns, prefixe list and postfix list

In [3]:


raw_postfix = [
        ('[\u200c](تر(ین?)?|گری?|های?)' , ''),
        (r'(تر(ین?)?|گری?)(?=$)' , ''), #تر ، ترین، گر، گری در آخر کلمه
        (r'(?<=[^ او])(م|ت|ش|مان|تان|شان)$' , ''), # حذف شناسه های مالکیت و فعل در آخر کلمه
        (r'(ان|ات|گی|گری|های)$' , ''), #ان، ات، ها، های آخر کلمه  
]

raw_arabic_notation = [
    # remove FATHATAN, DAMMATAN, KASRATAN, FATHA, DAMMA, KASRA, SHADDA, SUKUN
    ('[\u064B\u064C\u064D\u064E\u064F\u0650\u0651\u0652]', ''),
    ]

raw_long_letters = [
    (r' +', ' '),  # remove extra spaces
    (r'\n\n+', '\n'),  # remove extra newlines
    (r'[ـ\r]', ''),  # remove keshide, carriage returns
    ]

raw_half_space = [
    ('[\u200c]', ''),
]
 

## verbs, mokassar,morakab list

In [4]:
present_roots =['توان','باش','رو','بر','یاور', 'یانداز', 'یای','یاندیش','بخش','باز','خر','بین','شنو','دار','دان','رسان','شناس','گو','گذار','یاب','لرز','ساز','شو','نویس','خوان','کاه','گیر','خواه','کن' ]

past_roots = ['توانست','بود','کرد','اورد','انداخت','امد','خرید','باخت','برد','رفت','اندیشید','بخشید','دید','شنید','داشت','دانست','رساند','شناخت','گفت','گذشت','یافت','لرزید','ساخت','شد','نوشت','خواند','کاست','گرفت','خواست']
all_verbs_roots = present_roots + past_roots
empty_list = ['','','','','','','']
verb_prefix = ['نمی‌', 'می‌','ن','ب',"" ]
present_verb_postfix = ['م','ی','د','ید','ند','یم']
past_verb_postfix = ['ایم','اید','ای','ام','اند']
past_verb_postfix2 = ['م','ی','ید','ند','یم']
present_verbs = []
past_verbs = []
all_verbs = {}
for pref in verb_prefix:
    for present_root, past_root in zip(present_roots, past_roots):
        for post in past_verb_postfix2:
            all_verbs[pref + past_root+post] = past_root
        for post in past_verb_postfix:
            all_verbs[past_root+'ه‌'+post] = past_root
        for post in present_verb_postfix:
            all_verbs[pref+present_root+post] = present_root
            
words_list = ['ادب', 'آداب', 'طرف', 'اطراف', 'حقیقت', 'حقایق', 'موج', 'امواج', 'ادیب', 'ادبا', 'عمق', 'اعماق', 'خزینه', 'خزائن', 'مرکز', 'مراکز', 'اثر', 'آثار', 'عالم', 'علما', 'خبر', 'اخبار', 'موقع', 'مواقع', 'اسیر', 'اسرا', 'علم', 'علوم', 'دوره', 'ادوار', 'مصرف', 'مصارف', 'اسم', 'اسامی', 'علامت', 'علائم', 'دین', 'ادیان', 'معرفت', 'معارف', 'اسطوره', 'اساطیر', 'علت', 'علل', 'دفتر', 'دفاتر', 'مبحث', 'مباحث', 'امیر', 'امرا', 'عقیده', 'عقائد', 'ذخیره', 'ذخایر', 'ماده', 'مواد', 'امر', 'اوامر', 'عمل', 'اعمال', 'رفیق', 'رفقا', 'مذهب', 'مذاهب', 'امام', 'ائمه', 'عید', 'اعیاد', 'رای', 'آرا', 'مصیبت', 'مصائب', 'اصل', 'اصول', 'عنصر', 'عناصر', 'رسم', 'رسوم', 'معبد', 'معابد', 'افق', 'آفاق', 'عاطفه', 'عواطف', 'رابطه', 'روابط', 'مسجد', 'مساجد', 'بیت', 'ابیات', 'عضو', 'اعضا', 'رمز', 'رموز', 'معبر', 'معابر', 'تاجر', 'تجار', 'عبارت', 'عبارات', 'رجل', 'رجال', 'مظهر', 'مظاهر', 'تصویر', 'تصاویر', 'عجیب', 'عجایب', 'رقم', 'ارقام', 'منظره', 'مناظر', 'جد', 'اجداد', 'فقیه', 'فقها', 'زاویه', 'زوایا', 'مرض', 'امراض', 'جانب', 'جوانب', 'فن', 'فنون', 'زعیم', 'زعما', 'مورد', 'موارد', 'جزیره', 'جزایر', 'فکر', 'افکار', 'سانحه', 'سوانح', 'مرحله', 'مراحل', 'جبل', 'جبال', 'فریضه', 'فرایض', 'سلطان', 'سلاطین', 'مفهوم', 'مفاهیم', 'جریمه', 'جرایم', 'فعل', 'افعال', 'شعر', 'اشعار', 'منبع', 'منابع', 'حادثه', 'حوادث', 'فقیر', 'فقرا', 'شاعر', 'شعرا', 'مکان', 'اماکن', 'حشم', 'احشام', 'قاعده', 'قواعد']
mokassar_dict ={}
for i in range(0,len(words_list),2):
    mokassar_dict[words_list[i+1]] = words_list[i]
                                       

## tokenizer

In [5]:
def split(txt, seps):
    default_sep = seps[0]    
    # we skip seps[0] because that's the default separator
    for sep in seps[1:]:
        txt = txt.replace(sep, default_sep)
    return [i.strip() for i in txt.split(default_sep)]

#get doc and return (word,doc id)
def tokenizer(text):
    for l in english_char:
        text = text.replace(l,"")
    word_list = split(text.strip().replace("\n"," "), sep_list)
    word_list = [x for x in word_list if not x.startswith("ir") and not x.startswith("NID")]
    return word_list

## normalizer

In [6]:
def remove_punctuation_marks(word_list):
#     word_list = list(word_list)
    for i in range(len(word_list)):
        for mark in punctuation_marks:
            word_list[i] = word_list[i].replace(mark,"")

    return word_list


def edit_long_letters(word_list):
    patterns_compiled = [(re.compile(x[0]), x[1]) for x in raw_long_letters]
    for pattern, rep in patterns_compiled:
        for i in range(len(word_list)):
            word_list[i] = pattern.sub(rep, word_list[i])
    return word_list




def remove_arabic_notation(word_list):
    patterns_compiled = [(re.compile(x[0]), x[1]) for x in raw_arabic_notation]
    for pattern, rep in patterns_compiled:
        for i in range(len(word_list)):
            word_list[i] = pattern.sub(rep, word_list[i])
    return word_list


def create_translation_table(src_list, dst_list):
     return dict((ord(a), b) for a, b in zip(src_list, dst_list)) #map src unicode to dst char

    
def char_digit_Unification(word_list): 
    word_list = [x for x in word_list if not x.isnumeric() or (x.isnumeric() and float(x)<3000)]
    translation_src, translation_dst = ' ىكي“”', ' یکی""'
    translation_src += 'ئ0123456789أإآ%'
    translation_dst += 'ی۰۱۲۳۴۵۶۷۸۹ااا٪'
    translations_table = create_translation_table(translation_src, translation_dst)
    for i in range(len(word_list)):
        word_list[i] = word_list[i].translate(translations_table)
    return word_list

def remove_mokassar(word_list):
    for i in range(len(word_list)):
        word_list[i] = mokassar_dict.get(word_list[i],word_list[i])
        
    return word_list
    


def verb_Steaming(word_list):
    for i in range(len(word_list)):
        word_list[i] = all_verbs.get(word_list[i],word_list[i])
        
    return word_list
    

def remove_postfix(word_list):

    patterns_compiled = [(re.compile(x[0]), x[1]) for x in raw_postfix]
    for pattern, rep in patterns_compiled:
        for i in range(len(word_list)):
            if len(word_list[i])>4 and  not (word_list[i] in all_verbs_roots):
                word_list[i] = pattern.sub(rep, word_list[i])
    return word_list


    
    
def remove_prefix(word_list):
    starts =['بی‌' , 'نا', 'با']
    for i in range(len(word_list)):
        for start in starts:
            if word_list[i].startswith(start) and len(word_list[i])>4:
                word_list[i] = word_list[i][len(start):]
                break

    return word_list

def morakab_Unification(word_list):
    patterns_compiled = [(re.compile(x[0]), x[1]) for x in raw_half_space]
    for pattern, rep in patterns_compiled:
        for i in range(len(word_list)):
            word_list[i] = pattern.sub(rep, word_list[i])
    return word_list

#all to gether 
def normalizer(word_list):
    #remove punc
    for i in range(len(word_list)):
        for mark in punctuation_marks:
            word_list[i] = word_list[i].replace(mark,"")
            
    #edit long letters
    patterns_compiled = [(re.compile(x[0]), x[1]) for x in raw_long_letters]
    for pattern, rep in patterns_compiled:
        for i in range(len(word_list)):
            word_list[i] = pattern.sub(rep, word_list[i])
            
    #remove mokassar
    for i in range(len(word_list)):
        word_list[i] = mokassar_dict.get(word_list[i],word_list[i])
    
    #remove_arabic_notation
    patterns_compiled = [(re.compile(x[0]), x[1]) for x in raw_arabic_notation]
    for pattern, rep in patterns_compiled:
        for i in range(len(word_list)):
            word_list[i] = pattern.sub(rep, word_list[i])
    
    #char_digit_Unification
    word_list = [x for x in word_list if not x.isnumeric() or (x.isnumeric() and float(x)<3000)]
    translation_src, translation_dst = ' ىكي“”', ' یکی""'
    translation_src += 'ئ0123456789أإآ%'
    translation_dst += 'ی۰۱۲۳۴۵۶۷۸۹ااا٪'
    translations_table = create_translation_table(translation_src, translation_dst)
    for i in range(len(word_list)):
        word_list[i] = word_list[i].translate(translations_table)
    
    #verb_Steaming
    for i in range(len(word_list)):
        word_list[i] = all_verbs.get(word_list[i],word_list[i])
        
    #remove_prefix
    starts =['بی‌' , 'نا', 'با']
    for i in range(len(word_list)):
        for start in starts:
            if word_list[i].startswith(start) and len(word_list[i])>4:
                word_list[i] = word_list[i][len(start):]
                break
    
    #remove_postfix
    patterns_compiled = [(re.compile(x[0]), x[1]) for x in raw_postfix]
    for pattern, rep in patterns_compiled:
        for i in range(len(word_list)):
            if len(word_list[i])>4 and  not (word_list[i] in all_verbs_roots):
                word_list[i] = pattern.sub(rep, word_list[i])
    #morakab_Unification
    patterns_compiled = [(re.compile(x[0]), x[1]) for x in raw_half_space]
    for pattern, rep in patterns_compiled:
        for i in range(len(word_list)):
            word_list[i] = pattern.sub(rep, word_list[i])
            
    for word in word_list:
        if word in stop_words:
            word_list.remove(word)
    
    return [x for x in word_list if len(x) >= 2]

## read data

In [7]:
%%time
all_data = pd.DataFrame()
pd_list = []
pd_list .append(pd.read_excel('IR00_3_11k News.xlsx', engine = 'openpyxl'))
pd_list .append(pd.read_excel('IR00_3_17k News.xlsx', engine = 'openpyxl'))
pd_list .append(pd.read_excel('IR00_3_20k News.xlsx', engine = 'openpyxl'))
# pd_list .append(pd.read_excel('3.xlsx', engine = 'openpyxl'))
# pd_list .append(pd.read_excel('2.xlsx', engine = 'openpyxl'))
# pd_list .append(pd.read_excel('1.xlsx', engine = 'openpyxl'))
for df in pd_list:
    all_data = all_data.append(df,ignore_index=True)
all_data["topic"].replace({"political": "politics", "sport": "sports"}, inplace=True)
all_data = all_data[all_data.content != 'None\n\n']
all_data = all_data.reset_index(drop=True)
all_data.index +=1
all_data['id'] = all_data.index
contents = all_data['content'].to_list()
doc_ids = all_data['id'].to_list()
doc_url = all_data['url']
docs_topic = all_data['topic']
docs_num = len(doc_ids)


Wall time: 8.59 s


## create inverted index and change docs content to bag of word

In [8]:

def create_dictionary(contents, doc_ids):
    for content, id in zip(contents,doc_ids):
        doc_tokens = tokenizer(content)
        doc_normalized_tokens = normalizer(doc_tokens)
        term_freq_dict = dict(collections.Counter(doc_normalized_tokens))
        
        for term, freq in term_freq_dict.items():
            term_freq_dict[term] = (1 + math.log((freq), 10))
        contents[id-1] = term_freq_dict
        for term,freq in term_freq_dict.items():
            term_postings = inverted_index.get(term,{})
            term_postings[id] = freq
            inverted_index[term] = term_postings

def calculate_tf_idf_weight(contents):
    docs_len = []
    for doc in contents:
        length = 0
        for term in doc.keys():
            df = len(inverted_index[term])
            idf = math.log(docs_num/df,10)
            doc[term] *= idf 
            length += doc[term]
        docs_len.append( math.sqrt(length))
    return docs_len

            
          


In [14]:
%%time
inverted_index ={}
create_dictionary(contents, doc_ids)
docs_len = calculate_tf_idf_weight(contents)



Wall time: 5min 36s


## save inverted_index and docs_len

In [15]:
%%time
file_inverted_index = open('ph3_inverted_index.obj', 'wb') 
pickle.dump(inverted_index, file_inverted_index)
file_inverted_index.close()
inverted_index = {}




Wall time: 4.6 s


In [16]:
file_docs_len = open('ph3_docs_len.obj', 'wb') 
pickle.dump(docs_len, file_docs_len)
file_docs_len.close()
docs_len = {}

In [17]:
file_contents = open('ph3_contents.obj', 'wb') 
pickle.dump(contents, file_contents)
file_contents.close()
contents =[]

In [19]:
# inverted_index['سلام']
# docs_len =[]

## read inverted_index and docs_len

In [9]:
%%time
readed = open('ph3_inverted_index.obj', 'rb') 
inverted_index = pickle.load(readed)
readed = open('docs_len.obj', 'rb') 
docs_len = pickle.load(readed)
readed = open('ph3_contents.obj', 'rb') 
contents = pickle.load(readed)

Wall time: 4.26 s


## Kmeans Clustring

In [10]:
def choose_random_centroids(k):
    centroid_docs_id = (np.random.uniform(0, docs_num, size = k).astype(int))
    centroids = { x:contents[x-1] for x in centroid_docs_id}
    return centroids

In [11]:
def assign_to_leader(centeroids,centroids_len, contents, first):
    
    clusters = {x:[] for x in centeroids.keys()}
    for doc_id in doc_ids:
        relevent_flag = False
        doc_content = contents[doc_id-1]
        max_Score = 0
        best_centroid = 0
        for centroid_id, centroid_content in centeroids.items():
            centroid_score = 0
            for term in doc_content.keys():
                centroid_score += (doc_content[term] * centroid_content.get(term, 0))
            
            if first == 0:
                centroid_score /= docs_len[centroid_id-1]
            else: 
                centroid_score /= centroids_len[centroid_id]
            
            if centroid_score> max_Score:
                max_Score = centroid_score
                best_centroid = centroid_id
                relevent_flag = True
        if relevent_flag:
            clusters[best_centroid].append(doc_id)
    return clusters
        

In [12]:
def avg_cluster(clusters, k):
    new_centroids = {-1*x:{} for x in range(1,k+1)}
    centroids_len = {-1*x:0 for x in range(1,k+1)}
    counter = -1
    for cluster_followers in clusters.values():
        new_centroid = {}
        cluster_sum = 0
        for doc_id in cluster_followers:
            for term,w_t in contents[doc_id-1].items():
                w = new_centroid.get(term,0)
                new_centroid[term] = w + w_t
                
        for key in new_centroid.keys():
            new_centroid[key] /= len(cluster_followers)
            cluster_sum += new_centroid[key]**2
        new_centroids[counter] = new_centroid
        centroids_len[counter] =math.sqrt(cluster_sum)
        counter -= 1
    return  new_centroids, centroids_len

In [13]:
def Kmeans_clustering(k, number_of_iteration):
    new_centroids = choose_random_centroids(k)
    clusters = []
    centroids_len = {}
    for i in tqdm(range(number_of_iteration)):
        clusters =  assign_to_leader(new_centroids,centroids_len ,contents, i)

        if i != number_of_iteration-1: 
            new_centroids, centroids_len = avg_cluster(clusters, k)

    return clusters, new_centroids, centroids_len


In [72]:
# %%time
# clusters, centroids, centroids_len = Kmeans_clustering(10,5)

In [16]:
%%time
clusters, centroids, centroids_len = Kmeans_clustering(100,5)

100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [19:52<00:00, 238.53s/it]

Wall time: 19min 52s


In [17]:
for key in clusters.keys():
    print(len(clusters[key]))

673
765
662
564
580
90
869
416
141
1254
506
576
187
262
819
240
1031
462
241
484
598
157
1156
482
258
308
247
384
853
445
136
617
645
322
304
565
824
364
528
745
680
150
482
613
869
1194
146
780
500
391
78
138
787
526
445
601
529
406
357
795
181
784
163
264
169
693
590
597
242
717
397
271
300
291
92
519
199
230
152
543
783
279
591
567
616
504
547
504
629
356
731
586
1047
354
485
455
585
439
521
540


## save clusters data

In [26]:
%%time
file_clusters = open('clusters.obj', 'wb') 
pickle.dump(clusters, file_clusters)
file_clusters.close()
clusters = {}

Wall time: 3.99 ms


In [27]:
file_centroids = open('centroids.obj', 'wb') 
pickle.dump(centroids, file_centroids)
file_centroids.close()
centroids = {}

In [28]:
file_centroids_len = open('centroids_len.obj', 'wb') 
pickle.dump(centroids_len, file_centroids_len)
file_centroids_len.close()
centroids_len = {}

## read clusters data

In [14]:
readed = open('clusters.obj', 'rb') 
clusters = pickle.load(readed)
readed = open('centroids.obj', 'rb') 
centroids = pickle.load(readed)
readed = open('centroids_len.obj', 'rb') 
centroids_len = pickle.load(readed)

## query responding

In [15]:
def vectorize_query(query_str):
    query_tokens = tokenizer(query_str)
    query_normalized_tokens = normalizer(query_tokens)
    term_freq_dict = dict(collections.Counter(query_normalized_tokens))
    query_terms = {}
    for  term,freq in term_freq_dict.items():
        if term in inverted_index.keys() and term not in stop_words:
            query_terms[term] = 1+math.log(freq,10)
    print(query_terms)
    return query_terms
    

In [16]:
def find_best_leader(centroids, centroids_len, query_terms, k,b):
    similarity_score = {-1*x:0 for x in range(1,k+1)}
    for term, w_q in query_terms.items():
        for centroid_id, centroid_content in centroids.items():
            similarity_score[centroid_id] += w_q * centroid_content.get(term, 0)
    for key in similarity_score.keys():
        similarity_score[key] /= centroids_len[key]
    return sorted(similarity_score, key=similarity_score.get, reverse=True)[:b]


In [17]:
def score_docs(query_terms, cluster_docs):
    docs_score = {x:0 for x in cluster_docs}
    for term,Wt_q in query_terms.items():
        for doc_id in cluster_docs:
            docs_score[doc_id] += Wt_q * contents[doc_id-1].get(term, 0)
    
    for doc_id in cluster_docs:
        docs_score[doc_id] /= docs_len[doc_id-1]
    return docs_score

In [20]:
def extract_top_k(docs_similarity, k):

    heap = [(-value, key) for key,value in docs_similarity.items()]
    largest = heapq.nsmallest(k, heap)
    top_k = [(y, -x) for x,y in largest]
    return top_k

In [23]:
def query_responding(clusters_num, b):
    query = input("enter query: ")
    query_terms = vectorize_query(query)
    best_leader_ids = find_best_leader(centroids, centroids_len, query_terms, clusters_num,b)
    print(best_leader_ids)
    cluster_docs = list(set(clusters[best_leader_ids[0]] + clusters[best_leader_ids[1]]))
    docs_similarity = score_docs(query_terms, cluster_docs)
    a = datetime.datetime.now()
    top_docs = extract_top_k(docs_similarity, 50)
    b = datetime.datetime.now()

    print("{:<5} result in {} ms\nid \t(score)\t\t  -> link\n".format(len(top_docs),1000*(b-a).total_seconds()))
    for doc_id, score in top_docs:
        print( "{:<5}({}) -> {} ".format(doc_id,"-", doc_url[doc_id]))

## test

In [26]:
query_responding(100,4)


enter query: اعزام کاروان های اردوی راهیان نور
{'اعزام': 1.0, 'کارو': 1.0, 'های': 1.0, 'اردوی': 1.0, 'راهی': 1.0, 'نور': 1.0}
[-32, -41, -15, -33]
50    result in 0.0 ms
id 	(score)		  -> link

9702 (-) -> https://www.isna.ir/news/98021106170/حضور-لرستانی-ها-در-آیین-افتتاحیه-اردوهای-راهیان-نور-غرب-کشور 
9934 (-) -> https://www.isna.ir/news/98032712899/حضور-دانشجویان-چهارمحال-و-بختیاری-در-یادمان-های-دفاع-مقدس 
9004 (-) -> https://www.isna.ir/news/99092821569/سردار-کارگر-ماهیت-راهیان-نور-نباید-تغییر-کند 
10656(-) -> https://www.isna.ir/news/98080904715/مراسم-افتتاحیه-راهیان-نور-دانش-آموزی-برگزار-می-شود 
10031(-) -> https://www.isna.ir/news/98041910370/خوزستانی-ها-به-مناطق-عملیاتی-غرب-کشور-می-روند 
10043(-) -> https://www.isna.ir/news/98041910370/خوزستانی-ها-به-مناطق-عملیاتی-غرب-کشور-می-روند 
10692(-) -> https://www.isna.ir/news/98081408553/اعلام-جزئیات-حضور-دانش-آموزان-گیلانی-در-اردوهای-راهیان-نور 
10835(-) -> https://www.isna.ir/news/98091309743/توضیحات-سردار-عبدالله-پور-درباره-بزرگترین